### Run the cells below and click on 'Refresh' button to start the demo.
Toggle Next Case and Previous Case buttons to explore the cases¶

In [11]:
# Cell: Setup inicial seguro para Jupyter
import os
import traceback

import lightkurve as lk
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown, display, clear_output
import ipywidgets as widgets

# helper para mostrar Markdown
def printmd(string):
    display(Markdown(string))

# Activa plots inline en Jupyter
%matplotlib inline

# --- Cargar functions.py de forma segura (si existe) ---
functions_namespace = {}
functions_path = 'functions.py'
if os.path.exists(functions_path):
    try:
        with open(functions_path, 'r', encoding='utf-8') as f:
            code = f.read()
        # Ejecutar en un namespace separado para evitar contaminación del global
        exec(compile(code, functions_path, 'exec'), functions_namespace)
        print(f"✔ functions.py cargado en 'functions_namespace' (llaves: {list(functions_namespace.keys())[:10]}...)")
    except Exception:
        print(f"✖ Error al ejecutar {functions_path}:")
        traceback.print_exc()
else:
    print(f"⚠ No se encontró {functions_path}. Continúa sin él (asegúrate de colocarlo en el mismo directorio si lo necesitas).")

# --- Función segura para leer pickles (evita romper si falta el archivo) ---
def safe_read_pickle(path):
    if os.path.exists(path):
        try:
            df = pd.read_pickle(path)
            print(f"✔ Cargado: {path}  — shape: {getattr(df, 'shape', 'unknown')}")
            return df
        except Exception:
            print(f"✖ Error leyendo {path}:")
            traceback.print_exc()
            return None
    else:
        print(f"⚠ No se encontró: {path}")
        return None

# --- Cargar los dataframes que necesitas ---
case_interest_lc = safe_read_pickle('./data/case_interest_lc.pkl')
predicted_planets_shaps = safe_read_pickle('./data/predicted_planets_shaps.pkl')

# Si no existen, dejar DataFrames vacíos para no romper el resto del notebook
if case_interest_lc is None:
    case_interest_lc = pd.DataFrame()
if predicted_planets_shaps is None:
    predicted_planets_shaps = pd.DataFrame()


✔ functions.py cargado en 'functions_namespace' (llaves: ['__builtins__', 'pandas', 'plot_selected', 'show_global_view', 'show_local_view', 'plot_shap_values', 'next_button_clicked', 'prev_button_clicked', 'refresh_button_clicked']...)
✔ Cargado: ./data/case_interest_lc.pkl  — shape: (181, 2001)
✔ Cargado: ./data/predicted_planets_shaps.pkl  — shape: (3724, 715)


In [12]:
# Cell: Configuración de widgets interactivos
import ipywidgets as widgets
from IPython.display import display

# --- Crear botones y elementos de UI ---
next_button = widgets.Button(
    description='Next Case',
    disabled=False,
    button_style='success',
    layout=widgets.Layout(width='50%')
)
prev_button = widgets.Button(
    description='Previous Case',
    disabled=False,
    button_style='danger',
    layout=widgets.Layout(width='50%')
)
dropdown = widgets.Dropdown(
    options=['Global View', 'Folded View', 'SHAP Values'],
    value='Global View',
    description='Data Type:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)
refresh_button = widgets.Button(
    description='Refresh',
    disabled=False,
    button_style='info',
    layout=widgets.Layout(width='50%')
)
out = widgets.Output(layout={'border': '1px solid black', 'padding': '10px'})
ix = 0  # índice del caso actual

# --- Asignar callbacks de forma segura ---
def safe_connect(button, func_name):
    """Conecta el botón a la función si existe en functions_namespace."""
    func = functions_namespace.get(func_name)
    if callable(func):
        button.on_click(func)
        print(f"✔ Callback '{func_name}' conectado a {button.description}")
    else:
        print(f"⚠ No se encontró '{func_name}' en functions.py — {button.description} sin acción.")

safe_connect(refresh_button, 'refresh_button_clicked')
safe_connect(next_button, 'next_button_clicked')
safe_connect(prev_button, 'prev_button_clicked')

# --- Construcción de la interfaz ---
uphbox = widgets.HBox([dropdown, refresh_button])
buttons = widgets.HBox([prev_button, next_button])
ui = widgets.VBox([buttons, uphbox, out])

display(ui)


✔ Callback 'refresh_button_clicked' conectado a Refresh
✔ Callback 'next_button_clicked' conectado a Next Case
✔ Callback 'prev_button_clicked' conectado a Previous Case
